In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [2]:
train = pd.read_csv('./output/train.csv',nrows=50000)    
test = pd.read_csv('./output/test.csv')    
test2 = pd.read_csv('./output/test2.csv')

def within_measure(x, y):
    #  3750901.5068 ≤ 𝑥 ≤ 3770901.5068
    #  −19268905.6133 ≤ 𝑦 ≤ −19208905.6133
    if 3750901.5068 <= x and x <= 3770901.5068 and -19268905.6133 <= y and y <= -19208905.6133:
        return 1
    else:
        return 0

train = train.append(test[test.x_exit==test.x_exit])
columns=[#'hash', 'trajectory_id', 
         'time_entry', 'time_exit', 'vmax', 'vmin',
       'vmean', 'x_entry', 'y_entry', 'first', 'last',
       'x_req', 'y_req', 'x_start', 'y_start', 'distance', 'time_start',
       'time_delta', 'time_total', 'city_start', 'city_now', 'city_was',
       'x_dir', 'y_dir', 'len',
         'distance_to_city', 'speed',
        #'x_exit', 'y_exit', 
        ]
x_train = train[columns]
x_test = test2[columns]




y_train=pd.DataFrame()

y_train['x_exit']=train['x_exit']
y_train['y_exit']=train['y_exit']

In [3]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219422 entries, 0 to 202935
Data columns (total 2 columns):
x_exit    219422 non-null float64
y_exit    219422 non-null float64
dtypes: float64(2)
memory usage: 5.0 MB


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 219422 entries, 0 to 202935
Data columns (total 29 columns):
hash                219422 non-null object
trajectory_id       219422 non-null object
time_entry          219422 non-null int64
time_exit           219422 non-null int64
vmax                70199 non-null float64
vmin                70199 non-null float64
vmean               73673 non-null float64
x_entry             219422 non-null float64
y_entry             219422 non-null float64
x_exit              219422 non-null float64
y_exit              219422 non-null float64
first               219422 non-null bool
last                219422 non-null bool
x_req               219422 non-null float64
y_req               219422 non-null float64
x_start             219422 non-null float64
y_start             219422 non-null float64
distance            219422 non-null float64
time_start          219422 non-null int64
time_delta          219422 non-null int64
time_total          219422 n

In [ ]:
from keras.models import Sequential
from keras.layers import SimpleRNN
from keras.layers import LSTM
model = Sequential()
from keras.layers import Dense

X_train = np.reshape(x_train.values, x_train.values.shape + (1,))
X_test = np.reshape(x_test.values, x_test.values.shape + (1,))

size_nn = np.shape(x_train)[1]

model.add(LSTM(10, input_shape=X_train.shape[1:]))
#model.add(Dense(units=50, activation='relu', input_dim=size_nn))
model.add(Dense(units=100, activation='relu'))
#model.add(Dense(units=1000, activation='relu'))
#model.add(SimpleRNN( units=10, activation='tanh', 

model.add(Dense(units=2, activation='softmax'))

from keras import optimizers

sgd = optimizers.SGD(lr=0.1, clipnorm=1.)

model.compile(loss='mean_squared_error',
              optimizer=sgd,
              metrics=['mean_squared_error'],
             )


devide_clusters = False


def nn():
    # batch_size=500
    
    #trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    #testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    
    model.fit(X_train, y_train.values, batch_size=32, epochs=10,verbose=True)
    
    #Prediction    
    target=list(model.predict(X_test))
    
    #print('Train Evaluation:',model.evaluate(x_train, y_train.values))

    return target
  
        

city_lgbm=pd.DataFrame()


result = nn()   
result = np.array(result)
city_lgbm['x_exit']=result[:,0]
city_lgbm['y_exit']=result[:,1]

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
219422/219422 [==============================] - 112s 510us/step - loss: nan - mean_squared_error: nan - loss: n
Epoch 2/10
219422/219422 [==============================] - 111s 505us/step - loss: nan - mean_squared_error: nan
Epoch 3/10
219422/219422 [==============================] - 106s 482us/step - loss: nan - mean_squared_error: nan
Epoch 4/10
107264/219422 [=============>................] - ETA: 55s - loss: nan - mean_squared_error: nan

In [ ]:




city = [within_measure(x, y) for x,y in zip(city_lgbm.x_exit.values,city_lgbm.y_exit.values)]
city_sqrt = [within_measure(x, y) for x,y in zip(test2.x_entry.values,test2.y_entry.values)]


In [ ]:






submission= pd.DataFrame()
trajectory_id = test2['trajectory_id']
submission['id']=trajectory_id 
submission['target'] = city
submission.to_csv('./output/nn/lstm.csv',index=False)    



In [ ]:
def plotmap(ycity,title):
    fig, ax = plt.subplots()
    fig.set_figheight(10)
    fig.set_figwidth(16)

    df=test2[['x_entry','y_entry']]
    df['sqrt']=city_sqrt
    df['target']=ycity
    #df=df[df.sqrt==0]
    leg=['Outside','City']
    # Plot
    for data in [df[df.target==1]]:
        ax.scatter(data.x_entry.values, data.y_entry.values,c='r',s=10, alpha=0.5,label=leg.pop(0))

    # Create a Rectangle patch
    rect = patches.Rectangle((3750901.5068,-19268905.6133),3770901.5068-3750901.5068,19268905.6133-19208905.6133,linewidth=2,edgecolor='y',facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect)    

    ax.set(xlabel='x', ylabel='y',
           title=title)
    ax.legend(loc="upper left", markerscale=2, scatterpoints=1, fontsize=10)
    plt.show()

In [ ]:
plotmap(city,'nn')

In [ ]:
if devide_clusters:
    plotmap(city_lgbm_c.target.values,'Clusters LGBM')

In [ ]:
if devide_clusters:
    plotmap(city.target.values,'Submission outside the sqrt')